In [1]:
1

1

In [ ]:
import numpy as np
import tensorflow.keras
from tensorflow import keras
import torch
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv1D, AveragePooling1D, Conv2D, MaxPooling2D,ReLU
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model #save and load models
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger, ModelCheckpoint
import IPython.display as ipd
from kymatio import Scattering1D
import hitdifferentparts
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import pescador
import random
import os
import librosa
import pickle

In [ ]:
# open J-8,Q=1,O=2 pkl file
J = 8
Q = 1
order = 2
pkl_path = '/scratch/hh2263/drum_data/pkl_data/scattering_J-08_Q-01_order2.pkl'
pkl_file = open(pkl_path, 'rb')
Sy_train,y_train = pickle.load(pkl_file) 

pkl_path_val = '/scratch/hh2263/drum_data/val/J_8_Q_1_order_2.pkl'
pkl_val = open(pkl_path_val,'rb')
Sy_val,y_val = pickle.load(pkl_val)
Sy_val = Sy_val.reshape((Sy_val.shape[2],Sy_val.shape[0],Sy_val.shape[1]))

In [ ]:
input_x = Sy_train.shape[1]
input_y = Sy_train.shape[2]
print(Sy_train.shape,Sy_val.shape,y_train.shape,y_val.shape,input_x,input_y)

In [ ]:
df_train = pd.read_csv("./train_param.csv")
df_test = pd.read_csv("./test_param.csv")
df_val = pd.read_csv("./val_param.csv")
df_full = pd.read_csv("./diffshapes_param.csv")

# normalization of the physical parameters
params = df_train.values[:,1:-1]
scaler = MinMaxScaler()
scaler.fit(params)

#normalize training and validation set
y_train_normalized = scaler.transform(y_train)
y_val_normalized = scaler.transform(y_val)

In [ ]:
#make the model
def create_model(J,Q,order,input_x,input_y,k_size,layer_size,nchan_out):
    #fname = random.choice(os.listdir(path_to_train))
    #rand_audio = os.path.join(path_to_train,fname)
    #y,sr = librosa.load(rand_audio)
    #N = len(y)
    #scattering = Scattering1D(J = J,shape=(N,), Q = Q, max_order=order)
    
    #Sy = getsc_new(torch.Tensor(y),J,Q,order,scattering).T
    #nrow, ncol = Sy.shape 
    nrow = input_x
    ncol = input_y
    #naudio = batch_size         # number of images in batch
    nchan_in = 1       # number of input channels.  1 since it is BW
    #input_shape = (nrow,ncol,nchan_in)  # input shape of 1 image
    input_shape = (input_x,input_y)#Sy.shape
    #batch_shape = (naudio,nrow,ncol,nchan_in)  # shape of image batch
    #x = Sy.reshape(batch_shape)
    kernel_size = (k_size,)
    #nchan_out = 16

    K.clear_session()
    model=Sequential()
    #1 conv layer +  1 batch normalization + nonlinear activation + pooling
    model.add(Conv1D(input_shape=input_shape, filters=nchan_out,
                     kernel_size=kernel_size, padding="same",name='conv1'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(AveragePooling1D(pool_size=(4,)))
    
   #second time
    model.add(Conv1D(filters=nchan_out,
                     kernel_size=kernel_size, padding="same",name='conv2' ))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(AveragePooling1D(pool_size=(4,)))
    
    #third time
    if layer_size>=3:
        model.add(Conv1D(filters=nchan_out,
                         kernel_size=kernel_size, padding="same",name='conv3' ))
        model.add(BatchNormalization())
        model.add(Activation("relu"))
        model.add(AveragePooling1D(pool_size=(4,)))
        if layer_size==4:
        #fourth time
            model.add(Conv1D(filters=nchan_out,
                             kernel_size=kernel_size, padding="same",name='conv4' ))
            model.add(BatchNormalization())
            model.add(Activation("relu"))
            model.add(AveragePooling1D(pool_size=(2,)))
            if layer_size ==5:
                model.add(Conv1D(filters=nchan_out,
                             kernel_size=kernel_size, padding="same",name='conv5' ))
                model.add(BatchNormalization())
                model.add(Activation("relu"))
                model.add(AveragePooling1D(pool_size=(2,)))

    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    #what activation should be chosen for last layer, for regression problem? should be a linear function
    model.add(Dense(5, activation='linear')) #output layer that corresponds to the 5 physical parameters.


    # Compile the model
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    
    #print(model.summary())
    return model

In [ ]:
model=create_model(J,Q,order,input_x,input_y,k_size=4,layer_size=4,nchan_out=8)

## stick with batch size 32+steps_per_epoch 10
find different model architecture
1. k_size
2. layer_size
3. nchan_out

In [8]:
#divide the training up to 1/8, use batch size of 64
k = [4,8,16]
layer = [3,4,5]
nout = [8,16]
n = Sy_train.shape[0]
steps_per_epoch = 10
bs = 32
m = bs*steps_per_epoch
idx = np.arange(0,n,1)
record = []
for k_s in k:
    for lay in layer:
        for no in nout:
            val_loss=[]
            loss=[]
            model=create_model(J,Q,order,input_x,input_y,k_size=k_s,layer_size=lay,nchan_out=no)
            for epoch in range(30):
                np.random.shuffle(idx)
                Sy_temp = Sy_train[idx[:m],:,:]
                y_temp = y_train_normalized[idx[:m],:]
                hist = model.fit(Sy_temp,
                            y_temp,
                            epochs=1,
                            verbose=2,
                            batch_size=bs,
                            validation_data = (Sy_val,y_val_normalized),
                            use_multiprocessing=True)
                loss.append(hist.history['loss'])
                val_loss.append(hist.history['val_loss'])
            record.append((k_s,lay,no,min(loss),min(val_loss),np.where(np.array(val_loss)==min(val_loss))))


Train on 320 samples, validate on 7776 samples
320/320 - 10s - loss: 1.0108 - mean_squared_error: 1.0108 - val_loss: 0.1340 - val_mean_squared_error: 0.1340
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.4481 - mean_squared_error: 0.4481 - val_loss: 0.1047 - val_mean_squared_error: 0.1047
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.2860 - mean_squared_error: 0.2860 - val_loss: 0.0856 - val_mean_squared_error: 0.0856
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.2000 - mean_squared_error: 0.2000 - val_loss: 0.0710 - val_mean_squared_error: 0.0710
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.1742 - mean_squared_error: 0.1742 - val_loss: 0.0581 - val_mean_squared_error: 0.0581
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.1428 - mean_squared_error: 0.1428 - val_loss: 0.0476 - val_mean_squared_error: 0.0476
Train on 320 samples, validate on 7776 samples
320/320 - 1s - l

Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0536 - mean_squared_error: 0.0536 - val_loss: 0.3777 - val_mean_squared_error: 0.3777
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0491 - mean_squared_error: 0.0491 - val_loss: 0.3799 - val_mean_squared_error: 0.3799
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0509 - mean_squared_error: 0.0509 - val_loss: 0.3999 - val_mean_squared_error: 0.3999
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0562 - mean_squared_error: 0.0562 - val_loss: 0.4333 - val_mean_squared_error: 0.4333
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0539 - mean_squared_error: 0.0539 - val_loss: 0.4522 - val_mean_squared_error: 0.4522
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0506 - mean_squared_error: 0.0506 - val_loss: 0.4838 - val_mean_squared_error: 0.4838
Train on 320 samples, validate on 7776 samples
320/320 - 1s - lo

Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0571 - mean_squared_error: 0.0571 - val_loss: 0.0501 - val_mean_squared_error: 0.0501
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0544 - mean_squared_error: 0.0544 - val_loss: 0.0511 - val_mean_squared_error: 0.0511
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0635 - mean_squared_error: 0.0635 - val_loss: 0.0540 - val_mean_squared_error: 0.0540
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0554 - mean_squared_error: 0.0554 - val_loss: 0.0596 - val_mean_squared_error: 0.0596
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0550 - mean_squared_error: 0.0550 - val_loss: 0.0687 - val_mean_squared_error: 0.0687
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0605 - mean_squared_error: 0.0605 - val_loss: 0.0702 - val_mean_squared_error: 0.0702
Train on 320 samples, validate on 7776 samples
320/320 - 1s - lo

Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0576 - mean_squared_error: 0.0576 - val_loss: 0.0729 - val_mean_squared_error: 0.0729
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0681 - mean_squared_error: 0.0681 - val_loss: 0.0820 - val_mean_squared_error: 0.0820
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0721 - mean_squared_error: 0.0721 - val_loss: 0.0891 - val_mean_squared_error: 0.0891
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0582 - mean_squared_error: 0.0582 - val_loss: 0.0991 - val_mean_squared_error: 0.0991
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0582 - mean_squared_error: 0.0582 - val_loss: 0.1112 - val_mean_squared_error: 0.1112
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0544 - mean_squared_error: 0.0544 - val_loss: 0.1104 - val_mean_squared_error: 0.1104
Train on 320 samples, validate on 7776 samples
320/320 - 1s - lo

Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.1804 - mean_squared_error: 0.1804 - val_loss: 0.0921 - val_mean_squared_error: 0.0921
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.1471 - mean_squared_error: 0.1471 - val_loss: 0.0811 - val_mean_squared_error: 0.0811
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.1085 - mean_squared_error: 0.1085 - val_loss: 0.0720 - val_mean_squared_error: 0.0720
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0906 - mean_squared_error: 0.0906 - val_loss: 0.0639 - val_mean_squared_error: 0.0639
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0737 - mean_squared_error: 0.0737 - val_loss: 0.0598 - val_mean_squared_error: 0.0598
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0714 - mean_squared_error: 0.0714 - val_loss: 0.0568 - val_mean_squared_error: 0.0568
Train on 320 samples, validate on 7776 samples
320/320 - 1s - lo

Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0647 - mean_squared_error: 0.0647 - val_loss: 0.0632 - val_mean_squared_error: 0.0632
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0539 - mean_squared_error: 0.0539 - val_loss: 0.0734 - val_mean_squared_error: 0.0734
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0512 - mean_squared_error: 0.0512 - val_loss: 0.0863 - val_mean_squared_error: 0.0863
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0603 - mean_squared_error: 0.0603 - val_loss: 0.1017 - val_mean_squared_error: 0.1017
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0522 - mean_squared_error: 0.0522 - val_loss: 0.1262 - val_mean_squared_error: 0.1262
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 1.2409 - mean_squared_error: 1.2409 - val_loss: 0.1352 - val_mean_squared_error: 0.1352
Train on 320 samples, validate on 7776 samples
320/320 - 1s - lo

Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0497 - mean_squared_error: 0.0497 - val_loss: 0.1983 - val_mean_squared_error: 0.1983
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0553 - mean_squared_error: 0.0553 - val_loss: 0.2570 - val_mean_squared_error: 0.2570
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0556 - mean_squared_error: 0.0556 - val_loss: 0.3309 - val_mean_squared_error: 0.3309
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0517 - mean_squared_error: 0.0517 - val_loss: 0.4308 - val_mean_squared_error: 0.4308
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0466 - mean_squared_error: 0.0466 - val_loss: 0.5548 - val_mean_squared_error: 0.5548
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0509 - mean_squared_error: 0.0509 - val_loss: 0.7568 - val_mean_squared_error: 0.7568
Train on 320 samples, validate on 7776 samples
320/320 - 1s - lo

Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0649 - mean_squared_error: 0.0649 - val_loss: 0.0676 - val_mean_squared_error: 0.0676
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0655 - mean_squared_error: 0.0655 - val_loss: 0.0782 - val_mean_squared_error: 0.0782
Train on 320 samples, validate on 7776 samples
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0503 - mean_squared_error: 0.0503 - val_loss: 0.4320 - val_mean_squared_error: 0.4320
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0570 - mean_squared_error: 0.0570 - val_loss: 0.5036 - val_mean_squared_error: 0.5036
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0502 - mean_squared_error: 0.0502 - val_loss: 0.6167 - val_mean_squared_error: 0.6167
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.0512 - mean_squared_error: 0.0512 - val_loss: 0.8484 - val_mean_squared_error: 0.8484
Train on 320 samp

In [9]:
record

[(4,
  3,
  8,
  [0.047170025482773784],
  [0.0372450783496163],
  (array([8]), array([0]))),
 (4,
  3,
  16,
  [0.04907047338783741],
  [0.12679117507159465],
  (array([2]), array([0]))),
 (4,
  4,
  8,
  [0.04896475598216057],
  [0.028780571794436303],
  (array([11]), array([0]))),
 (4,
  4,
  16,
  [0.05008333809673786],
  [0.03699441645830992],
  (array([10]), array([0]))),
 (4,
  5,
  8,
  [0.05047358311712742],
  [0.027383882606041774],
  (array([12]), array([0]))),
 (4,
  5,
  16,
  [0.04549816995859146],
  [0.03641178824559406],
  (array([4]), array([0]))),
 (8,
  3,
  8,
  [0.04426582045853138],
  [0.05143693928058746],
  (array([6]), array([0]))),
 (8,
  3,
  16,
  [0.042094594426453115],
  [0.05528342397308644],
  (array([8]), array([0]))),
 (8,
  4,
  8,
  [0.05116790011525154],
  [0.03318405470239773],
  (array([9]), array([0]))),
 (8,
  4,
  16,
  [0.04322846084833145],
  [0.021089329495412822],
  (array([10]), array([0]))),
 (8,
  5,
  8,
  [0.0465620893985033],
  [0.062

## best model being - kernel size 8, layer 4, nchan_out 16, batchsize 32, steps_per_epoch 10 - save model as chkpt

train on different J,Q, order and save performance


In [18]:
model=create_model(J,Q,order,input_x,input_y,k_size=8,layer_size=4,nchan_out=16)
val_loss = []
for epoch in range(30):
    np.random.shuffle(idx)
    Sy_temp = Sy_train[idx[:m],:,:]
    y_temp = y_train_normalized[idx[:m],:]
    
    hist = model.fit(Sy_temp,
                y_temp,
                epochs=1,
                verbose=2,
                batch_size=32,
                validation_data = (Sy_val,y_val_normalized),
                callbacks = callbacks
                use_multiprocessing=True)
    
    val_loss.append(hist.history['val_loss'])
    if epoch >0 and hist.history['val_loss']>val_loss[-2]:
        model.save(model_filepath)
        np.savetxt(log_filepath, np.array(hist.history[]), delimiter=",")
             

Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 1.1009 - mean_squared_error: 1.1009 - val_loss: 0.1425 - val_mean_squared_error: 0.1425
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.3401 - mean_squared_error: 0.3401 - val_loss: 0.1213 - val_mean_squared_error: 0.1213
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.2241 - mean_squared_error: 0.2241 - val_loss: 0.1056 - val_mean_squared_error: 0.1056
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.1710 - mean_squared_error: 0.1710 - val_loss: 0.0940 - val_mean_squared_error: 0.0940
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.1508 - mean_squared_error: 0.1508 - val_loss: 0.0839 - val_mean_squared_error: 0.0839
Train on 320 samples, validate on 7776 samples
320/320 - 1s - loss: 0.1208 - mean_squared_error: 0.1208 - val_loss: 0.0751 - val_mean_squared_error: 0.0751
Train on 320 samples, validate on 7776 samples
320/320 - 1s - lo

ValueError: Expected 1D or 2D array, got 0D array instead

In [20]:
#dict = {'Python' : '.py', 'C++' : '.cpp', 'Java' : '.java'}
f = open(log_filepath,"w")
f.write( str(hist.history) )
f.close()


## ugh in order to enable save checkpoint i have to use generator

In [13]:
class DataGenerator(tensorflow.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, Sy_train, y_train, batch_size=32, shuffle=True):
        'Initialization'
        self.batch_size = batch_size
        self.Sy_train = Sy_train
        self.y_train = y_train
        self.shuffle = shuffle
        #self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(self.Sy_train.shape[0] / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = np.arange(0,self.Sy_train.shape[0],1)
        np.random.shuffle(indexes)
        
        # Generate data
        X, y = self.__data_generation(indexes[:self.batch_size])

        return X, y

#    def on_epoch_end(self):
#        'Updates indexes after each epoch'
#        self.indexes = np.arange(len(self.list_IDs))
#        if self.shuffle == True:
#            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = self.Sy_train[list_IDs_temp,:,:]
        y = self.y_train[list_IDs_temp,:]

        return X, y

In [14]:
params = {'batch_size': 64,
          'shuffle': True}
training_generator = DataGenerator(Sy_train, y_train_normalized, **params)

In [19]:
output_dir = "../output/"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
model_filepath = os.path.join(output_dir, "cp_best.ckpt")
log_filepath = os.path.join(output_dir, 'train_log.csv')

callbacks = []
callbacks.append(EarlyStopping(monitor='val_loss',patience=0,verbose=1))
callbacks.append(ModelCheckpoint(model_filepath, save_best_only=True))
callbacks.append(CSVLogger(log_filepath))

In [18]:
model=create_model(J,Q,order,input_x,input_y,k_size=8,layer_size=4,nchan_out=16)
model.fit(training_generator,
          steps_per_epoch=10,
          verbose=2,
          callbacks = callbacks,
          epochs = 10, 
         validation_data=(Sy_val,y_val_normalized),
          use_multiprocessing=True)





Epoch 1/10
Epoch 1/10
7776/10 - 1s - loss: 0.1792 - mean_squared_error: 0.1266
10/10 - 3s - loss: 1.0260 - mean_squared_error: 1.0260 - val_loss: 0.1270 - val_mean_squared_error: 0.1266
Epoch 2/10
Epoch 1/10
7776/10 - 1s - loss: 0.1347 - mean_squared_error: 0.0902
10/10 - 1s - loss: 0.3806 - mean_squared_error: 0.3806 - val_loss: 0.0906 - val_mean_squared_error: 0.0902
Epoch 3/10
Epoch 1/10
7776/10 - 1s - loss: 0.1018 - mean_squared_error: 0.0646
10/10 - 1s - loss: 0.2411 - mean_squared_error: 0.2411 - val_loss: 0.0649 - val_mean_squared_error: 0.0646
Epoch 4/10
Epoch 1/10
7776/10 - 1s - loss: 0.0762 - mean_squared_error: 0.0461
10/10 - 1s - loss: 0.1723 - mean_squared_error: 0.1723 - val_loss: 0.0464 - val_mean_squared_error: 0.0461
Epoch 5/10
Epoch 1/10
7776/10 - 1s - loss: 0.0589 - mean_squared_error: 0.0347
10/10 - 1s - loss: 0.1330 - mean_squared_error: 0.1330 - val_loss: 0.0349 - val_mean_squared_error: 0.0347
Epoch 6/10
Epoch 1/10
7776/10 - 1s - loss: 0.0464 - mean_squared_error

save checkpoint

In [20]:
model2=create_model(J,Q,order,input_x,input_y,k_size=8,layer_size=4,nchan_out=16)
model2.load_weights(model_filepath)

In [29]:
example = df_test.values[0,:]
name = str(example[0])+"_sound.wav"
params = example[1:]
y,fs = librosa.load("/scratch/hh2263/drum_data/test/"+name)

In [30]:
params 

array([12353.875421473127, 0.1515511077684384, 0.0015630271691019885,
       0.047278615741408986, 0.3968901393586017, 'test'], dtype=object)

In [31]:
scattering = Scattering1D(J=J, shape=(2**15,), Q=Q, max_order=order)
Sx = np.array(scattering(torch.Tensor(y)).T)

In [32]:
Sx.shape

(128, 43)

In [36]:
predicted = model2.predict(Sx.reshape((1,128,43)))

In [38]:
scaler.inverse_transform(predicted)

array([[6.1175972e+03, 1.2335956e-01, 3.6207625e-01, 2.7610660e-01,
        5.6290185e-01]], dtype=float32)